# 1. Comparing Sentinel-1 RTC datasets

Now that we have constructed and organized data cubes for both the ASF and Planetary Computer datasets, it would be useful and informative to compare the two. This notebook will demonstrate steps to ensure a direct comparison between the two datasets as well as use of xarray plotting tools for a preliminary visual comparison.

## Learning goals

**Xarray and python techniques:**
- Conditional selection based on non-dimensional coordinates using `xr.Dataset.where()`
- Subsetting datasets based on dimensional coordinates using `xr.DataArray.isin()`
- Adding dimensional and non-dimensional coordinates to `xr.Dataset` objects
- Xarray plotting methods
- Projecting xarray objects to different grids using `xr.interp_like()`

**High-level science goals**
- Comparing and evaluating multiple datasets
- Setting up multiple datasets for direct comparisons
- Handling differences in spatial resolution 

In [ ]:
%xmode minimal
import dask.distributed
import matplotlib.pyplot as plt
import numpy as np
import stackstac
import xarray as xr
import pystac
import planetary_computer

from dask.distributed import Client

import s1_tools

%matplotlib inline

## Overview

While the datasets we will be comparing in this notebook are quite similar, they contain important differences related to how they were processed. The product description pages for both datasets contain important information for understanding how they are generated.

### Source data
One important difference is the source data used to generate the RTC images. ASF uses the Single Look Complex ([SLC](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/single-look-complex)) images that contains both amplitude and phase information for each pixel, is in radar coordinates and has not yet been multi-looked. The Microsoft Planetary Computer RTC imagery is generate from ground range detected images (GRD). [GRD](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/ground-range-detected) data has been detected, multi-looked and projected to ground range.

### DEM
A digital elevation model (DEM) is an important input parameter for RTC processing. ASF RTC processing uses the Copernicus DEM with 30 m resolution. Planetary Computer uses higher resolution DEMs. One effect of the use of different DEMs during RTC processing is the different spatial resolutions of the RTC products: **Planetary Computer RTC imagery has a spatial resolution of 10 meters while ASF RTC imagery has a spatial resolution of 30 meters.**

### Read prepared data



The PC data cube is a bit too large (~ 1GB). rather than write it to disk like the other examples, maybe just recreate it in this nb? or go back to using cell magic to store it? if recreate it here, wrap in script

In [ ]:
time_range = "2021-01-01/2022-08-01"
bbox = [88.214935, 27.92767, 88.302, 28.034]
epsg = 32645

ds_pc = s1_tools.S1PC_DataCube(time_range, bbox, epsg)

In [4]:
from pystac_client import Client

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
time_range = "2021-01-01/2022-08-01"
bbox = [88.214935, 27.92767, 88.302, 28.034]

bbox_coords = s1_tools.points2coords(bbox)
search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range)
items = search.item_collection()




In [ ]:
from dask.distributed import Client
dask_client = Client(processes=False)
print(dask_client.dashboard_link)

da = stackstac.stack(planetary_computer.sign(items), bounds_latlon=bbox, epsg=32645)

stac_item = pystac.read_file(
    "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/S1A_IW_GRDH_1SDV_20210602T120544_20210602T120609_038161_0480FD_rtc"
)

def extract_source_granule_pc(rtc_id):
    base_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/"
    full_url = base_url + str(rtc_id)
    stac_item = pystac.read_file(full_url)
    source_granule = stac_item.links[5].target[-62:]
    return source_granule

granule_ls = [
    extract_source_granule_pc(da.isel(time=t).id.values) for t in range(len(da.time))
]
def make_granule_coord_pc(granule_ls):
    """this fn takes a list of granule IDs, extracts acq date for each granule, organizes this as an array that can be assigned as a coord to an xr obj"""

    acq_date = [pd.to_datetime(granule[17:25]) for granule in granule_ls]

    granule_da = xr.DataArray(
        data=granule_ls,
        dims=["time"],
        coords={"time": acq_date},
        attrs={
            "description": "source granule ID S1 GRD files used to process PC RTC images, extracted from STAC metadata"
        },
        name="granule_id",
    )

    return granule_da

granule_coord = make_granule_coord_pc(granule_ls)

da.coords["granule_id"] = ("time", granule_coord.data)

ds_pc = da.to_dataset(dim="band")


In [8]:
pc_cube = ds_pc

In [6]:
#pc_cube = xr.open_dataset("../data/tutorial2/s1_pc_cube.zarr", engine="zarr")

In [7]:
asf_cube = xr.open_dataset("../data/tutorial2/s1_asf_cube_updated.zarr", engine="zarr")

Read in the ASF dataset, which we named `vrt_full` in the earlier notebook:

In [ ]:
asf_cube

Before we go further, the nodata value for the ASF dataset is currently zero. Change all zero values to NaN:

In [9]:
asf_cube = asf_cube.where(asf_cube.vv != 0.0, np.nan, drop=False)

In [ ]:
asf_cube

## Extract common data take ID from granule IDs

We want to ensure that we are performing a direct comparison of the ASF and PC datasets. To do this, we would like to use the acquisition ID that is stored in the source granule name (published by ESA). In the setup notebooks we attached the entire granule IDs of the SLC images to the ASF dataset and the GRD images to the PC dataset. In the ASF data inspection notebook, we attached data take id as a non-dimensional coordinate. Now we will do the same for the Planetary Computer dataset, extracting just the 6-digit acquisition ID from the granule ID and using this for a scene-by-scene comparison.

In [11]:
data_take_pc = [
    str(pc_cube.isel(time=t).granule_id.values)[56:] for t in range(len(pc_cube.time))
]

Assign `data_take_id` as a non-dimensional coordinate. Rather than use the `xr.assign_coords()` function, we are adding the object by specifying a tuple with the form ('coord_name', coord_data, attrs):

In [12]:
pc_cube.coords["data_take_id"] = ("time", data_take_pc)

In [ ]:
asf_cube

In [ ]:
pc_cube

### Subset to only common data takes

Now we have data take ID coordinates for both datasets. We want to find the common data take IDs between the two datasets. To do this, I extract a list of the acquisition IDs (`data_take_id`) for both datasets and then find the intersection of the two lists (the list object `common_data_takes`)

In [15]:
pc_data_take_ls = list(pc_cube.data_take_id.values)
asf_data_take_ls = list(asf_cube.data_take_id.values)

In [ ]:
common_data_takes = list(set(pc_data_take_ls) & set(asf_data_take_ls))
len(common_data_takes)

It looks like there are 83 RTC images that are generated from common acquisitions between the two datasets

We'd like to subset the ASF and the PC datasets to only the common acquisitions. The xarray `.isin()` method is very useful for this type of selection. 

In [17]:
subset_condition_asf = asf_cube.data_take_id.isin(common_data_takes)

subset_condition_pc = pc_cube.data_take_id.isin(common_data_takes)

Next, we want to select the elements along the `acq_ate` (and `time`) dimensions where `data_take_id` satisfies the requirement of being in the `common_data_takes` list. I found this stack overflow answer useful for this step: [https://stackoverflow.com/questions/70777676/xarray-select-dataarray-according-to-an-non-dimension-coordinate](https://stackoverflow.com/questions/70777676/xarray-select-dataarray-according-to-an-non-dimension-coordinate).

In [43]:
asf_subset = asf_cube.sel(acq_date=asf_cube.data_take_id.isin(common_data_takes))

In [ ]:
asf_subset.acq_date

In [45]:
pc_subset = pc_cube.sel(time=pc_cube.data_take_id.isin(common_data_takes))

In [ ]:
pc_subset["vv"].encoding

In [47]:
pc_subset = pc_subset.chunk({"time": 18, "y": 193, "x": 111})

## Visual comparison

Let's perform some preliminary visual comparisons of the datasets:

#### Individual time steps

Use `asf_bc_sidebyside()` to compare snapshot views of backscatter at individual time steps:

In [ ]:
pc_subset

In [ ]:
s1_tools.asf_pc_sidebyside(asf_subset, pc_subset, 10)

In [ ]:
s1_tools.asf_pc_sidebyside(asf_subset, pc_subset, 4)

### Average backscatter over time

Now we will look at the backscatter mean along the time dimension

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

s1_tools.power_to_db(asf_subset.vv.mean(dim=["x", "y"])).plot(
    ax=ax, color="blue", linestyle="None", marker="o", markersize=5, label="ASF"
)
s1_tools.power_to_db(pc_subset.vv.mean(dim=["x", "y"])).plot(
    ax=ax, color="red", linestyle="None", marker="o", markersize=5, label="PC"
)
fig.legend();

## Handling spatial resolution differences

The PC dataset has a higher spatial resolution than the ASF dataset, which you can see in the sizes of the x and y dimensions of the dataset. The ASF dataset masks out more pixels due to shadow than the PC dataset. To explore whether or not this is the cause of the observed backscatter offset between the two, we need to mask out the pixels in the PC dataset that are masked in the ASF dataset. 

In order to do this, first we must downsample the PC dataset to match the spatial resolution of the ASF dataset. There are multiple xarray methods that could be used but here we will use the `xr.interp_like()` method because we would like the PC x and y dimensions to match the ASF dimensiosn exactly. 

In [52]:
asf_subset = asf_subset.rename({"acq_date": "time"})

In [ ]:
asf_subset.time

In [54]:
pc_downsample = pc_subset.interp_like(asf_subset)

In [ ]:
pc_downsample

In [56]:
pc_mask = xr.where(asf_subset[["vh", "vv"]].notnull(), pc_downsample, np.nan)

In [ ]:
pc_mask

Oops, this isn't what we want. Now we have a 5-dimensional object instead of a 4-dimensional object, because the dimensions of the `asf_subset` object were broadcast onto the `pc_downsample` object, meaning that we have both a `time` and a `acq_date` object.

Let's go back and try to fix this...

In [ ]:
pc_downsample.isel(time=0).time.values

In [ ]:
asf_subset.isel(time=0).time.values

Okay, so we need to take the time off of the `pc_downsample` time coordinates:

In [60]:
pc_downsample.coords["time"] = pc_downsample.time.dt.date

In [ ]:
pc_downsample.coords.dtypes

In [ ]:
pc_subset.time

In [ ]:
asf_subset.time

In [ ]:
xr.__version__

## TODO
This didn't throw an error before but does now (xarray: `'2024.10.0'`), fix later


In [ ]:
asf_subset.coords.dtypes

Now let's try again:

In [ ]:
pc_mask = xr.where(asf_subset.notnull(), pc_downsample, np.nan)

In [ ]:
pc_mask

Great! Looks like its working

In [ ]:
asf_subset

In [ ]:
pc_mask

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

s1_tools.power_to_db(asf_subset.vv.mean(dim=["x", "y"])).plot(
    ax=ax, color="blue", linestyle="None", marker="o", markersize=5, label="ASF"
)
s1_tools.power_to_db(pc_mask["vv"].mean(dim=["x", "y"], skipna=True)).plot(
    x="time", ax=ax, color="red", linestyle="None", marker="o", markersize=5, label="PC"
);

## Break up by asc, desc pass


Use `xr.where()` to subset datasets by orbital pass:

In [ ]:
asf_subset

In [47]:
asf_desc = asf_subset.where(asf_subset["orbital_dir"] == "desc", drop=True)
asf_asc = asf_subset.where(asf_subset["orbital_dir"] == "asc", drop=True)

In [ ]:
# asf_asc.vv.mean(dim=['x','y']).plot(marker='o', linestyle='None')

In [51]:
pc_subset = pc_subset.compute()

In [52]:
pc_desc = pc_subset.where(pc_subset["sat:orbit_state"] == "descending", drop=True)
pc_asc = pc_subset.where(pc_subset["sat:orbit_state"] == "ascending", drop=True)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16, 8))

s1_tools.power_to_db(asf_desc.vv.mean(dim=["x", "y"])).plot(
    ax=axs[0],
    color="blue",
    linestyle="None",
    marker="+",
    markersize=5,
    label="ASF VV desc",
)
s1_tools.power_to_db(pc_desc.vv.mean(dim=["x", "y"])).plot(
    ax=axs[0],
    color="red",
    linestyle="None",
    marker="+",
    markersize=5,
    label="PC VV desc",
)
s1_tools.power_to_db(asf_asc.vv.mean(dim=["x", "y"])).plot(
    ax=axs[0],
    color="blue",
    linestyle="None",
    marker="o",
    markersize=5,
    label="ASF VV asc",
)
s1_tools.power_to_db(pc_asc.vv.mean(dim=["x", "y"])).plot(
    ax=axs[0],
    color="red",
    linestyle="None",
    marker="o",
    markersize=5,
    label="PC VV asc",
)
axs[0].legend(loc="lower right")
axs[0].set_xlabel("Acquisition date")
axs[0].set_ylabel("Backscatter (sigma-nought) (dB)")
axs[0].set_title("VV")

s1_tools.power_to_db(asf_desc.vh.mean(dim=["x", "y"])).plot(
    ax=axs[1],
    color="blue",
    linestyle="None",
    marker="+",
    markersize=5,
    label="ASF VH desc",
)
s1_tools.power_to_db(pc_desc.vh.mean(dim=["x", "y"])).plot(
    ax=axs[1],
    color="red",
    linestyle="None",
    marker="+",
    markersize=5,
    label="PC VH desc",
)
s1_tools.power_to_db(asf_asc.vh.mean(dim=["x", "y"])).plot(
    ax=axs[1],
    color="blue",
    linestyle="None",
    marker="o",
    markersize=5,
    label="ASF VH asc",
)
s1_tools.power_to_db(pc_asc.vh.mean(dim=["x", "y"])).plot(
    ax=axs[1],
    color="red",
    linestyle="None",
    marker="o",
    markersize=5,
    label="PC VH asc",
)
axs[1].legend(loc="lower right")
axs[1].set_xlabel("Acquisition date")
axs[1].set_ylabel("Backscatter (sigma-nought) (dB)")
axs[1].set_title("VH")

fig.suptitle(
    "Comparison of backscatter values from ASF-processed and PC-processed Sentinel-1 RTC imagery"
);